In [68]:
import pandas as pd
import OpenDartReader
import FinanceDataReader as fdr
import datetime
from bs4 import BeautifulSoup
from pykrx import stock
import math

In [69]:
api_key = '133579488f1e7f9a44707b2a7d2be9999840935a'
dart = OpenDartReader(api_key)

In [70]:
day_buy = '2021-04-14'

In [71]:
list_d002 = dart.list(end=day_buy, kind='D', kind_detail='D002')
list_d002.head()

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00145437,키위미디어그룹,012170,Y,임원ㆍ주요주주특정증권등소유상황보고서,20210414002310,추준호,20210414,
1,00781202,뉴프라이드,900100,K,의결권대리행사권유참고서류,20210414002307,뉴프라이드,20210414,
2,00778235,TS인베스트먼트,246690,K,임원ㆍ주요주주특정증권등소유상황보고서,20210414002285,변기수,20210414,
3,00141273,웰바이오텍,010600,Y,주식등의대량보유상황보고서(일반),20210414002283,대양디엔아이,20210414,
4,00778235,TS인베스트먼트,246690,K,임원ㆍ주요주주특정증권등소유상황보고서,20210414002275,조경훈,20210414,


In [72]:
list_report_nm = list_d002[(list_d002['report_nm'] == '임원ㆍ주요주주특정증권등소유상황보고서') & ((list_d002['corp_cls'] == 'Y') | (list_d002['corp_cls'] == 'K' ))]
list_report_nm.head()

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00145437,키위미디어그룹,012170,Y,임원ㆍ주요주주특정증권등소유상황보고서,20210414002310,추준호,20210414,
2,00778235,TS인베스트먼트,246690,K,임원ㆍ주요주주특정증권등소유상황보고서,20210414002285,변기수,20210414,
4,00778235,TS인베스트먼트,246690,K,임원ㆍ주요주주특정증권등소유상황보고서,20210414002275,조경훈,20210414,
5,00778235,TS인베스트먼트,246690,K,임원ㆍ주요주주특정증권등소유상황보고서,20210414002260,티에스인베스트홀딩스,20210414,
7,00778235,TS인베스트먼트,246690,K,임원ㆍ주요주주특정증권등소유상황보고서,20210414002250,김웅,20210414,


In [73]:
result = pd.DataFrame(columns=['corp_name', 'corp_code', 'mdf_stk_cnt'])
for idx, row in list_report_nm.iterrows():
    html = dart.document(row['rcept_no'])
    soup = BeautifulSoup(html, 'html5lib')
    sec3_2 = soup.select('section-3')[2]
    rpt_rsn = sec3_2.findAll('tu', {'aunit': 'RPT_RSN'})[-1]
    mdf_stk_cnt = sec3_2.findAll('te', {'acode': 'MDF_STK_CNT'})[-1]
    if (rpt_rsn.contents[0] == '장내매수(+)'):
        tmp = pd.DataFrame(data=[[row['corp_name'], row['corp_code'], row['stock_code'], mdf_stk_cnt.contents[0]]], columns=['corp_name', 'corp_code', 'stock_code', 'mdf_stk_cnt'])
        result = result.append(tmp, ignore_index=True)
result

,corp_name,corp_code,mdf_stk_cnt,stock_code
0,아이비김영,01414422,400,339950
1,미래에셋맵스리츠,01437292,"9,761",357250
2,삼성공조,00126201,"2,000",006660
3,큐브엔터,00985686,"2,500",182360
4,브이티지엠피,00150165,"13,431",018290
5,바이오리더스,00587925,"10,000",142760
6,한화시스템,00339391,"1,778",272210
7,LG유플러스,00231363,"25,000",032640


In [74]:
finstate = dart.finstate('00155276', 2020, reprt_code='11014')
finstate.set_index(['fs_nm','account_nm']).head()

rcept_no reprt_code bsns_year corp_code stock_code  \
fs_nm  account_nm                                                             
연결재무제표 유동자산        20201113000671      11014      2020  00155276     003670   
       비유동자산       20201113000671      11014      2020  00155276     003670   
       자산총계        20201113000671      11014      2020  00155276     003670   
       유동부채        20201113000671      11014      2020  00155276     003670   
       비유동부채       20201113000671      11014      2020  00155276     003670   

                  fs_div sj_div  sj_nm   thstrm_nm      thstrm_dt  \
fs_nm  account_nm                                                   
연결재무제표 유동자산          CFS     BS  재무상태표  제 50 기3분기말  2020.09.30 현재   
       비유동자산         CFS     BS  재무상태표  제 50 기3분기말  2020.09.30 현재   
       자산총계          CFS     BS  재무상태표  제 50 기3분기말  2020.09.30 현재   
       유동부채          CFS     BS  재무상태표  제 50 기3분기말  2020.09.30 현재   
       비유동부채         CFS     BS  재무상태표  제 50 기3분기말  2020.09.30 현재   

                       thstrm_amount frmtrm_nm      frmtrm_dt  \
fs_nm  account_nm                                               
연결재무제표 유동자산          906,122,359,371   제 49 기말  2019.12.31 현재   
       비유동자산       1,157,219,860,943   제 49 기말  2019.12.31 현재   
       자산총계        2,063,342,220,314   제 49 기말  2019.12.31 현재   
       유동부채          183,499,608,768   제 49 기말  2019.12.31 현재   
       비유동부채         869,915,888,037   제 49 기말  2019.12.31 현재   

                       frmtrm_amount ord thstrm_add_amount frmtrm_add_amount  
fs_nm  account_nm                                                             
연결재무제표 유동자산          649,644,634,524   1               NaN               NaN  
       비유동자산       1,080,487,930,571   3               NaN               NaN  
       자산총계        1,730,132,565,095   5               NaN               NaN  
       유동부채          250,654,203,754   7               NaN               NaN  
       비유동부채         473,160,418,105   9               NaN               NaN

In [75]:
# "11013"=1분기보고서,  "11012"=반기보고서,  "11014"=3분기보고서, "11011"=사업보고서
result2 = pd.DataFrame()
for idx, row in result.iterrows():
    for year in [2018, 2019, 2020]:
        finstate = dart.finstate(row['corp_code'], year, reprt_code='11011')
        col_name = '당기순이익(' + str(year) + ')'
        if finstate is None:
            row[col_name] = '0'
        elif finstate[finstate['fs_nm'] == '연결재무제표'].empty:
            row[col_name] = finstate.set_index(['fs_nm','account_nm']).loc[('재무제표', '당기순이익'), 'thstrm_amount']
        else:
            try:
                row[col_name] = finstate.set_index(['fs_nm','account_nm']).loc[('연결재무제표', '당기순이익'), 'thstrm_amount']
            except:
                row[col_name] = '0'
    
    finstate = dart.finstate(row['corp_code'], 2020, reprt_code='11011')
    if finstate is None:
        row['부채총계'] = '0'
        row['자산총계'] = '0'
    elif finstate[finstate['fs_nm'] == '연결재무제표'].empty:
        row['부채총계'] = finstate.set_index(['fs_nm','account_nm']).loc[('재무제표', '부채총계'), 'thstrm_amount']
        row['자산총계'] = finstate.set_index(['fs_nm','account_nm']).loc[('재무제표', '자산총계'), 'thstrm_amount']
    else:
        try:
            row['부채총계'] = finstate.set_index(['fs_nm','account_nm']).loc[('연결재무제표', '부채총계'), 'thstrm_amount']
            row['자산총계'] = finstate.set_index(['fs_nm','account_nm']).loc[('연결재무제표', '자산총계'), 'thstrm_amount']
        except:
            row['부채총계'] = '0'
            row['자산총계'] = '0'
    result2 = result2.append(row)
result2

,corp_code,corp_name,mdf_stk_cnt,stock_code,당기순이익(2018),당기순이익(2019),당기순이익(2020),부채총계,자산총계
0,01414422,아이비김영,400,339950,0,0,"-759,422,591","30,957,462,846","59,472,719,941"
1,01437292,미래에셋맵스리츠,"9,761",357250,0,0,"-775,358,536","199,481,788,469","296,808,413,266"
2,00126201,삼성공조,"2,000",006660,"6,332,358,995","5,039,088,673","5,388,558,395","35,616,240,675","258,204,690,029"
3,00985686,큐브엔터,"2,500",182360,"1,101,720,145","986,622,964","-624,538,048","21,633,078,732","58,849,477,346"
4,00150165,브이티지엠피,"13,431",018290,"10,707,739,301","3,411,551,246","-1,308,655,827","88,499,146,070","158,931,246,112"
5,00587925,바이오리더스,"10,000",142760,"-16,288,372,674","-41,657,220,099","-33,414,027,746","97,405,364,280","190,693,154,865"
6,00339391,한화시스템,"1,778",272210,0,"72,896,109,675","93,580,544,300","1,662,387,743,462","2,695,070,123,421"
7,00231363,LG유플러스,"25,000",032640,"481,609,000,000","438,831,000,000","478,065,000,000","10,698,976,000,000","18,350,190,000,000"


In [76]:
market_cap = stock.get_market_cap_by_ticker(day_buy.replace('-',''))
print(market_cap.head())

            종가             시가총액       거래량           거래대금       상장주식수
티커                                                                  
005930   84000  501461734200000  13794652  1156326758700  5969782550
000660  137000   99736324005000   2777203   381902748000   728002365
035420  391500   64309119142500    757969   298405770500   164263395
051910  897000   63321331671000    555279   493200725000    70592343
005935   75100   61798791170000   1159732    87222340600   822886700


In [77]:
market_cap.loc['000660', '시가총액']

99736324005000

In [78]:
result3 = pd.DataFrame()
for idx, row in result2.iterrows():
    row['순이익가치'] = str(math.floor((int(row['당기순이익(2020)'].replace(',', '')) * 3 + int(row['당기순이익(2019)'].replace(',', '')) * 2 + int(row['당기순이익(2018)'].replace(',', '')) * 1) / 6 * 10))
    row['순자산'] = str(int(row['자산총계'].replace(',', '')) - int(row['부채총계'].replace(',', '')))
    row['기업가치'] = str(math.floor(int(row['순이익가치']) * 0.6 +  int(row['순자산']) * 0.4))
    row['시가총액'] = str(market_cap.loc[row['stock_code'], '시가총액'])
    row['가치/시총'] = '{:.2f}%'.format(int(row['기업가치']) / int(row['시가총액']) * 100)
    result3 = result3.append(row)
result3

,corp_code,corp_name,mdf_stk_cnt,stock_code,가치/시총,기업가치,당기순이익(2018),당기순이익(2019),당기순이익(2020),부채총계,순이익가치,순자산,시가총액,자산총계
0,01414422,아이비김영,400,339950,8.75%,9127835065,0,0,"-759,422,591","30,957,462,846",-3797112955,28515257095,104304738025,"59,472,719,941"
1,01437292,미래에셋맵스리츠,"9,761",357250,37.01%,36604574310,0,0,"-775,358,536","199,481,788,469",-3876792680,97326624797,98892000000,"296,808,413,266"
2,00126201,삼성공조,"2,000",006660,146.00%,121611591267,"6,332,358,995","5,039,088,673","5,388,558,395","35,616,240,675",54293685876,222588449354,83294718500,"258,204,690,029"
3,00985686,큐브엔터,"2,500",182360,11.78%,16087911374,"1,101,720,145","986,622,964","-624,538,048","21,633,078,732",2002253215,37216398614,136522004290,"58,849,477,346"
4,00150165,브이티지엠피,"13,431",018290,14.35%,41777714328,"10,707,739,301","3,411,551,246","-1,308,655,827","88,499,146,070",22674790520,70432100042,291039118920,"158,931,246,112"
5,00587925,바이오리더스,"10,000",142760,-113.58%,-162529779876,"-16,288,372,674","-41,657,220,099","-33,414,027,746","97,405,364,280",-333074826850,93287790585,143091767520,"190,693,154,865"
6,00339391,한화시스템,"1,778",272210,37.80%,839606804233,0,"72,896,109,675","93,580,544,300","1,662,387,743,462",710889753750,1032682379959,2221142338350,"2,695,070,123,421"
7,00231363,LG유플러스,"25,000",032640,102.35%,5853951599999,"481,609,000,000","438,831,000,000","478,065,000,000","10,698,976,000,000",4655776666666,7651214000000,5719608829100,"18,350,190,000,000"
